In [1]:
import numpy as np
import pandas as pd
import itertools
import random
import time
from concurrent.futures import ProcessPoolExecutor

In [2]:
# generate the model parameters
class bayesian_network_posterior:
    def __init__(self, the_R, tau2_R, kap_R, lam_R, mu, num_state, num_action, n_time, beta, v2):
        self.the_R = the_R
        self.tau2_R = tau2_R
        self.kap_R = kap_R
        self.lam_R = lam_R
        self.num_nodes = (num_state + num_action) * n_time
        self.num_state = num_state
        self.num_action = num_action
        self.mu = mu
        self.beta = beta
        self.v2 = v2
        self.n_time = n_time

    def posterior_sample(self, size=1, useFixed = True):
        p_beta = np.zeros(shape=(self.num_nodes,self.num_nodes, size))
        p_v2 = np.zeros(shape=(self.num_nodes, size))
        for i in range(self.num_nodes):
            for j in range(self.num_nodes):
                if self.tau2_R[i, j] != 0:
                    p_beta[i, j, ] = np.random.normal(loc=self.the_R[i,j], scale=np.sqrt(self.tau2_R[i,j]), size=size)
            gamma_rate = self.lam_R[i] / 2
            p_v2[i,] = 1 / np.random.gamma(shape=self.kap_R[i] / 2, scale=1/gamma_rate, size=size)
        if useFixed:
            p_beta = self.beta
            p_v2 = self.v2
        return (p_beta, p_v2, self.mu)

class bayesian_network:
    def __init__(self, mu, beta, v2, num_action, num_state, n_time, normalized = True, sample_mean=None, sample_sd=None):
        self.n_time = n_time
        if normalized:
            self.sample_mean = sample_mean
            self.sample_sd = sample_sd
            self.normalized = True
        self.initial_state_full = np.array([0.05, 0.00, 0.00, 30.00, 5.00,0.7])
        if num_state == 4:
            self.initial_state_base = self.initial_state_full[[0, 1, 3, 4, 5]]
        if num_state == 5:
            self.initial_state_base = self.initial_state_full
        self.mu = mu
        self.v2 = v2
        self.beta = beta
        self.num_action = num_action
        self.num_state = num_state
        self.n_factor = num_action + num_state
        self.beta_state = np.zeros(shape=(n_time, num_state, num_state)) # s -> s
        self.beta_action = np.zeros(shape=(n_time, num_action, num_state)) # a -> s
        for i in range(n_time-1):
            self.beta_state[i,:,:] = beta[(self.n_factor * (i+1) - num_state):(self.n_factor * (i+1)), (self.n_factor * (i + 2) - num_state): (self.n_factor * (i + 2))]
            self.beta_action[i,:,:] = beta[(self.n_factor * i):(self.n_factor * i + self.num_action), (self.n_factor * (i + 2) - num_state):(self.n_factor * (i + 2))]
        self.mu_a = []
        for i in range(self.n_time):
            temp_list = []
            for j in range(self.num_action):
                temp_list.append(self.mu[i * self.n_factor + j])
            self.mu_a.append(temp_list)

    def initial_state_generator(self, scale=10):
        init_states = self.initial_state_base + np.abs(np.random.normal(0, np.array(self.initial_state_base)/scale + 0.01))
        init_states = init_states[[0,1,3,4,5]] # init_states[:-1] * init_states[-1]
        self.initial_state = (init_states - self.sample_mean[(self.n_factor - self.num_state):self.n_factor]) / self.sample_sd[(self.n_factor - self.num_state):self.n_factor]

    def rescale_action(self, action, t, scale_method = "standard"):
        if not self.normalized:
            return action
        if scale_method == "standard":
            return self.sample_sd[(self.n_factor * t):(self.n_factor * t + self.num_action)] * action + self.sample_mean[(self.n_factor * t):(self.n_factor * t + self.num_action)]
        else:
            return self.sample_sd[(self.n_factor * t):(self.n_factor * t + self.num_action)] * action

    def rescale_state(self, state, t, scale_method = "standard"):
        if not self.normalized:
            return state
        if scale_method == "standard":
            return self.sample_sd[(self.n_factor * (t+1) - self.num_state):(self.n_factor * (t+1))] * state + self.sample_mean[(self.n_factor * (t+1) - self.num_state):(self.n_factor * (t+1))]
        else:
            return self.sample_sd[(self.n_factor * (t+1) - self.num_state):(self.n_factor * (t+1))] * state
        
beta_gibbs = pd.read_csv('data/beta_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
v2_gibbs = pd.read_csv('data/v2_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None,
                        dtype=np.float64)
the_R = pd.read_csv('data/the_R_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
tau2_R = pd.read_csv('data/tau2_R_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
kap_R = pd.read_csv('data/kap_R_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
lam_R = pd.read_csv('data/lam_R_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
mu = pd.read_csv('data/mu_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
sd = pd.read_csv('data/sd_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
bn_post = bayesian_network_posterior(the_R.to_numpy(), tau2_R.to_numpy(), kap_R.to_numpy(), lam_R.to_numpy(),
                                         mu.to_numpy().flatten(), 5, 1, 36, beta_gibbs, v2_gibbs)
# mua = np.array(mu.to_numpy().reshape(36, 6)[:, 0]).reshape(1, 36)
# mus = np.array(mu.to_numpy().reshape(36, 6)[:, 1:]).reshape(5, 36)

# 这个函数可以生成5个状态和5个状态之间的协方差矩阵，随机生成相关性矩阵，代表了H=36个周期
def simulate(H = 36, nums = 5, seed = 1):
    np.random.seed(seed)
    V_mat = np.zeros((H*nums, H*nums))
    mat = (2*np.random.rand((H-1)*nums, (H-1)*nums)-1)/5
    V_mat[nums*1:nums*H, nums*1:nums*H] = mat.dot(mat.T)
    return V_mat

def new_simulate(H = 36, nums = 5, seed = 1):
    np.random.seed(seed)
    V_mat = np.zeros((H*nums, H*nums))
    mat = (2*np.random.rand(H*nums, H*nums)-1)/5
    V_mat[nums*0:nums*H, nums*0:nums*H] = mat.dot(mat.T)
    return V_mat

In [3]:
np.random.seed(1)
nums = 5
H = 3
# 只需要生成一个样本，用来当作真实的参数
ssize1 = 1
#p_beta, p_v2, mu0 = bn_post.posterior_sample(ssize1, useFixed=False)
#v_mat = simulate(H=H, seed = 4) # 这个可以得到一个随机的random factor的协方差矩阵,定义为多元正态分布，均值为0
p_beta = np.load('p_beta.npy')
p_v2 = np.load('p_v2.npy')
mu0 = np.load('mu0.npy')
v_mat = np.load('v_mat.npy')
mu = pd.read_csv('data/mu_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64) 
s1 = np.array([0.05,0,30,5,0.7])
theta = np.load('theta.npy')

# b_r 和 c_r是组成奖励函数的参数
b_r = np.repeat(-534.52, H)
c_r = np.zeros((5, H))
c_r[1, H-1] = 1.29


beta = p_beta[:,:,0]
v2 = p_v2[:,0]
bn = bayesian_network(mu0, beta, v2, 1, 5, 36, True, mu, sd.to_numpy().flatten())
# mu_a mu_s beta_a beta_s theta_0事先就可以确定下来，不需要在过程中计算
mua = np.array(bn.mu_a).reshape(1, 36)
mus = np.array(bn.mu.reshape(36, 6)[:,1:]).reshape(5,36)
betas = np.zeros((5, 5, 36))
for i in range(36):
    betas[:,:,i] = bn.beta_state[i,:,:]
betaa = np.transpose(bn.beta_action.reshape(36,5))

# 为了减少维数，进行截断
mua = mua[:,:H]
mus = mus[:,:H]
betas = betas[:,:,:H]
betaa = betaa[:,:H]
theta = theta[:,:H-1]

theta_0 = np.concatenate((theta, np.zeros((nums, 1))), axis = 1)


In [5]:
# 定义一些基本的函数
def reward(b_r, c_r, a, s):
    return np.dot(b_r,a) + np.dot(c_r, s)

def policy(mua, mus, theta, s):
    return mua + np.dot(theta, s - mus)

# 这里的转移函数暂时不考虑随机因子
def transition(mus1, mus, mua, betas, betaa,s, a):
    return mus1 + np.dot(betas, s - mus) + betaa*(a - mua)

def get_elements_before(lst, element):
    # 获取目标元素在列表中的索引
    if element in lst:
        index = lst.index(element)
        # 返回索引之前的所有元素
        return lst[:index]
    else:
        return []  # 如果目标元素不在列表中，返回空列表

def factorial_iterative(n):
    result = 1
    for i in range(1, n + 1):
        result *= i
    return result

In [6]:
# 基于排列采样的shapley值的计算，首先是根据subset来采样随机因子
def random_factor_generator(subset):
    # 已知正态分布，根据subset中的随机因子采样
    # 获得subset中对应的方差和协方差信息
    global v_mat
    cov = np.zeros((len(subset), len(subset)))
    for i in range(len(subset)):
        for j in range(len(subset)):
            cov[i][j] = v_mat[subset[i]][subset[j]]
    mean = np.zeros(cov.shape[0])  # 均值向量，0向量
    sample = np.random.multivariate_normal(mean, cov, size=1)
    return sample[0]

def simulation_sample(subset, sample):
    # 根据随机因子和对应的值，采样
    global H, nums, v_mat
    # 为了生成条件分布下的样本，先得到条件分布
    rest_factor = [i for i in range(H*nums)]
    for i in subset:
        rest_factor.remove(i)
    
    x1 = sample
    
    sigma11 = np.zeros((len(subset), len(subset)))
    sigma12 = np.zeros((len(subset), len(rest_factor)))
    sigma22 = np.zeros((len(rest_factor), len(rest_factor)))
    for i in range(len(subset)):
        for j in range(len(subset)):
            sigma11[i][j] = v_mat[subset[i]][subset[j]]
    for i in range(len(subset)):
        for j in range(len(rest_factor)):
            sigma12[i][j] = v_mat[subset[i]][rest_factor[j]]
    for i in range(len(rest_factor)):
        for j in range(len(rest_factor)):
            sigma22[i][j] = v_mat[rest_factor[i]][rest_factor[j]]
    sigma21 = sigma12.T

    inverse_sigma11 = sigma11

    if np.linalg.det(sigma11) == 0: # 没有逆矩阵，使用伪逆
        inverse_sigma11 = np.linalg.pinv(sigma11)
    else:
        inverse_sigma11 = np.linalg.inv(sigma11)
    # 计算条件均值和条件协方差
    condition_mean = np.dot(sigma21, inverse_sigma11).dot(x1)
    condition_cov = sigma22 - np.dot(sigma21, inverse_sigma11).dot(sigma12)

    # 从条件分布中生成剩余3个变量的样本
    x2_sample = np.random.multivariate_normal(condition_mean, condition_cov, size = 1)
    return x2_sample[0]

def update_random_factor_sample(subset1,sample1, sample2):
    # 这个函数用来还原出随机因子的采样值
    global H, nums
    subset2 = [i for i in range(H*nums)]
    for i in subset1:
        subset2.remove(i)
    new_sample = np.zeros(H*nums)
    new_sample[subset1] = sample1
    new_sample[subset2] = sample2
    return new_sample

# 接下来是正式开始计算一次仿真的收益
def nn_value_function(subset):
    subset = sorted(subset)
    # 这里面是排好序的子集
    global H,nums,Nnn, v_mat
    if len(subset) == 0:
        res = 0
        mean = np.zeros(H*nums)
        for _ in range(Nnn):
            x = np.random.multivariate_normal(mean, v_mat, size = 3)
            y1 = PABN_simulation(x[0])
            y2 = PABN_simulation(x[1])
            y3 = PABN_simulation(x[2])
            res += y1*(y2-y3)
        return res/Nnn
    if len(subset) == H*nums:
        res = 0 
        for _ in range(Nnn):
            x0 = random_factor_generator(subset)
            x1 = random_factor_generator(subset)
            y1 = PABN_simulation(x0)
            y2 = PABN_simulation(x0)
            y3 = PABN_simulation(x1)
            res += y1*(y2-y3)
        return res/Nnn
    res = 0
    for _ in range(Nnn):
        # 随机变量的两个采样值
        xu0 = random_factor_generator(subset)
        xu1 = random_factor_generator(subset)
        xu0_rest1 = simulation_sample(subset, xu0)
        x1 = update_random_factor_sample(subset, xu0, xu0_rest1)
        xu0_rest2 = simulation_sample(subset, xu0)
        x2 = update_random_factor_sample(subset, xu0, xu0_rest2)
        xu1_rest = simulation_sample(subset, xu1)
        x3 = update_random_factor_sample(subset, xu1, xu1_rest)
        # 计算对应的reward
        y1 = PABN_simulation(x1)
        y2 = PABN_simulation(x2)
        y3 = PABN_simulation(x3)
        res += y1*(y2-y3)
    return res/Nnn

def PABN_simulation(random_factor):
    global H,mua, mus, theta_0 ,betaa, betas, b_r, c_r, s1
    cumulative_r = 0
    state = np.zeros((nums, H))
    state[:,0] = s1 + random_factor[0: nums]
    a = np.zeros(H)
    for t in range(H):
        if t == 0:
            state[:,t] = s1 + random_factor[0: nums]
        else:
            state[:,t] = transition(mus[:,t], mus[:,t-1], mua[:,t-1], betas[:,:,t-1], betaa[:,t-1], state[:,t-1], a[t-1]) + random_factor[t*nums: (t+1)*nums]
        a[t] = policy(mua[:,t], mus[:,t], theta_0[:,t], state[:,t])
        cumulative_r += reward(b_r[t], c_r[:,t], a[t], state[:,t])
        
    return cumulative_r


def random_permutations(iterable, n):
    # 创建排列的迭代器
    P_iterator = itertools.permutations(iterable)
    
    # 使用 reservoir sampling 算法来随机抽取n个排列
    result = []
    for i, perm in enumerate(P_iterator):
        if i < n:
            result.append(perm)
        else:
            # 在遍历过程中随机替换元素
            j = random.randint(0, i)  # 随机选择一个索引
            if j < n:
                result[j] = perm
    
    return result

# # 从40个元素的排列中随机获取10个排列
# result = random_permutations([i for i in range(20)], 10)

In [ ]:
# 这个部分专门用来并行计算，用于计算重复的n次实验得到的结果
s_time = time.time()
# 先用一个函数表示一次总的shapley计算，输入是重复的排列的个数
def shapley_value(N):
    global H,nums
    Sh_est = np.zeros(H*nums)
    Nnn = 1 # 表示求价值函数均值的样本量
    N0 = 40000 # 试点仿真下给每个随机因子都分配的样本数, 计算夏普利值N0*H*nums次
    #N = 100  总计算次数，二阶段分配次数 = N - N0*H*nums
    Sh2 = np.zeros(H*nums)
    N_set = np.zeros(H*nums)
    sigma_hat = np.zeros(H*nums)
    gHL = np.zeros(H*nums)
    z = -1.9599639845400545

    for _ in range(N0):
        perm = [i for i in range(H*nums)]
        random.shuffle(perm)
        for i in range(H*nums):
            N_set[i] += 1
            subset_before = get_elements_before(perm, i)
            subset_include = subset_before + [i]
            tmp = nn_value_function(subset_include) - nn_value_function(subset_before)
            Sh_est[i] += tmp
            Sh2[i] += tmp**2 

    # 根据试点结果计算样本标准差
    sigma_hat = np.sqrt((Sh2 +np.square(Sh_est)/N_set)/(N_set -1))

    for _ in range(N0*H*nums, N):
        gHL = np.abs(z)*sigma_hat/(2*(N_set)**(3/2))
        perm = [i for i in range(H*nums)]
        random.shuffle(perm)
        i_0 = np.argsort(gHL)[-1]
        N_set[i_0] += 1
        subset_before = get_elements_before(perm, i_0)
        subset_include = subset_before + [i_0]
        Sh_est[i_0] += nn_value_function(subset_include) - nn_value_function(subset_before)

    Sh_mean = Sh_est/N_set
    return Sh_mean

inputs = [1500000 for i in range(64)] # 第一个数是N的大小，第二个数是cpu数

with ProcessPoolExecutor(max_workers=64) as executor:
# 使用partial将fixed_param绑定到shapley_value函数
    results = list(executor.map(shapley_value, inputs))
    
e_time = time.time()
print('算法总耗时：', e_time-s_time)

算法总耗时： 3232.9941623210907
64
[ -29.78506267   56.59263524  -92.12529081 -141.19507966   52.98664477
  189.15746113  -45.68847094  589.36701717 -127.68790081   19.25636488
   49.02431733   -1.70976557   15.91361159  180.03826547 -132.23541469]


平均分配，一共15个因子，每个因子100个排列
一共1500次计算
N0=50 N=1500